In [ ]:
%matplotlib inline
from avidares.SingleExperiment import ResourceExperiment as RE
#from avidares.FactoredExperiment import FactoredResourceExperiment as FRE

from IPython.display import HTML
from avidares.BlitArtists import BRectangle, BCellHighlighter, BAnnotation
import matplotlib.pylab as plt
import pdb
import numpy as np

Single Resource Experiments
===============

This notebook demonstrates the avidares utilities to generate and plot a single experiment that uses resources.

There are two main classes: ResourceExperiment and ResourceExperimentAnimation.

ResourceExperiment performs the Avida experiment and loads the resource output file as a Pandas DataFrame.  It receives the following arguments:

    :param environment:  A string representation of the environment file.  Required.
    :param world_size:   A tuple of the (X,Y) size of the world.  Required.
    :param cwd:  The working directory to execute Avida.  Optional.
    :param args:  Arguments to pass to Avida aside from world size and location of input/output files.  Optional.
    :param evnets: The contents of the events file.  If not provided, a default is used. Optional
    :param use_pbar: Show the progress bar

All files created for the experiment are stored in a temporary directory.  This directory and its contents are deleted when the last reference to the ResourceExperiment is removed.

ResourceExperiment.animate() will make an animation of the resource abundances.  It may take up to three arguments:

    :param data_transform:  A function that will transform the resource data in the Pandas DataFrame
    :param figkw:  A dictionary of parameters to pass to the animation initializer method
    :param animkw: A dictionary of parameters to pass to the animation animate() method

In [ ]:
# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=1.0:outflow=0.5:xdiffuse=0.5:ydiffuse=0.5'
expr = RE(env, (60,60))
HTML(expr.run_experiment().animate().to_html5_video())

In [ ]:
# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=1.0:outflow=0.5:xdiffuse=1.0:ydiffuse=1.0:'
env += 'inflowx1=2:inflowx2=5:inflowy1=2:inflowy2=5:'
env += 'outflowx1=14:outflowx2=18:outflowy1=14:outflowy2=18'

r_in = BRectangle((2,2), 4, 4, ec='cyan', fc='none')
r_out = BRectangle((14,14), 5, 5, ec='magenta', fc='none')

figkw = {'title':'Inflow/Outflow Boxes','post_plot':[r_in, r_out]}

expr = RE(env, (20,20))
HTML(expr.run_experiment().animate(figkw=figkw).to_html5_video())

In [ ]:
env = 'RESOURCE res:geometry=grid:inflow=1.0:outflow=0.5:xdiffuse=1.0:ydiffuse=1.0:'
env += 'inflowx1=8:inflowx2=12:inflowy1=8:inflowy2=12'

# We are going to specify what events that we're going to use instead of relying
# on a provided default
events ='\
    u begin Inject default-heads.org\n\
    u 0:1:end PrintSpatialResources resources.dat\n\
    u 100 exit\n'

# Put a box around the inflow
r_in = BRectangle((8,8), 5, 5, ec='cyan', lw=3, fc='none')

# Some additional arguments to pass to the animation function
# figkw gets passed to the ResourceExperimentAnimation initializer
figkw = {'title':'Slow Motion Inflow Only','post_plot':[r_in], 'interval':1000}
# animkw gets passed to ResourceExperimentAnimation's animate method
animkw = {'fig_conf':{'figsize':(8,6), 'dpi':100}}

# Setup our expeririment
expr = RE(env, (20,20), events=events)

# Run, animate, and display our animation
HTML(expr.run_experiment().animate(figkw=figkw, animkw=animkw).to_html5_video())

In [ ]:
env = '\
RESOURCE res:geometry=torus:inflow=1.0:outflow=1.0:xdiffuse=1.0:ydiffuse=1.0:\\\n\
inflowx1=8:inflowx2=12:inflowy1=8:inflowy2=12:\\\n\
outflowx1=9:outflowx2=13:outflowy1=9:outflowy2=13\n'

# We are going to specify what events that we're going to use instead of relying
# on a provided default
events ='\
u begin Inject default-heads.org\n\
u 0:100:end PrintSpatialResources resources.dat\n\
u 10000 exit\n'

# Put a box around the inflow
r_in = BRectangle((8,8), 5, 5, ec='cyan', lw=3, fc='none')
r_out = BRectangle((9,9), 5, 5, ec='magenta', lw=3, fc='none')

# Some additional arguments to pass to the animation function
# figkw gets passed to the ResourceExperimentAnimation initializer
figkw = {
    'title':'inflow=1.0 outflow=1.0\nCyan=Inflow Magenta=Outflow',
    'post_plot':[r_in, r_out], 
    'interval':50,
    'env_string':env,
    'event_string':events}
# animkw gets passed to ResourceExperimentAnimation's animate method
animkw = {'fig_conf':{'figsize':(8,6), 'dpi':100}}

# Setup our expeririment
expr = RE(env, (20,20), events=events)

# Run, animate, and display our animation
html = expr.run_experiment().animate(figkw=figkw, animkw=animkw).to_html5_video()
plt.close()
HTML(html)


In [ ]:
env = '\
GRADIENT_RESOURCE resORN0:height=8:plateau=2:spread=15:common=1:\\\n\
updatestep=10:peakx=29:peaky=29:plateau_inflow=100:initial=1\
'

events ='\
u begin Inject default-heads.org\n\
u 0:200:end PrintSpatialResources resources.dat\n\
u 10000 exit\n'

class HighlightConsumable(BCellHighlighter):
    def __init__(self, gridshape, bdata):
        BCellHighlighter.__init__(self, gridshape, bdata=bdata)
    
    def blit_update(self, frame, update, ax_ndx=None, **kw):
        self.set_edgecolors('none')
        ec = []
        for ndx,cell_value in enumerate(self._bdata.iloc[frame,2:]):
            if cell_value >= 1.0:
                ec.append('yellow')
            else:
                ec.append('none')
        self.set_edgecolors(ec)
        return self



expr = RE(env, (60,60), events=events).run_experiment()

hl = HighlightConsumable((60,60), expr._data)
figkw = {
    'title':'Gradient Resource\nHighlighted Cells are Consumable',
    'interval':50,
    'env_string':env,
    'event_string':events,
    'post_plot':[hl]
}
animkw = {'fig_conf':{'figsize':(8,6), 'dpi':100}}

html = expr.animate(figkw=figkw, animkw=animkw).to_html5_video()
plt.close()
HTML(html)

In [ ]:
env = 'CELL cells:49..54,69..74,89..94:inflow=0.11:outflow=0.05:initial=0'

events ='\
u begin Inject default-heads.org\n\
u 0:1:end PrintSpatialResources resources.dat\n\
u 300  exit\n'


expr = RE(env, (20,20), events=events).run_experiment()

class AnnotateMaxAbundance(BAnnotation):
    def __init__(self, **kw):
        BAnnotation.__init__(self,**kw)
        
    def blit_update(self, frame, update, ax_ndx=None, **kw):
        max_amt = np.max(self._bdata.iloc[frame,2:])
        txt = 'Max Abundance = {0:.7f}'.format(max_amt)
        self.set_text(txt)


annot =\
    AnnotateMaxAbundance(
        bdata=expr._data,
        s='',xy=(12,5),xytext=(2,15), arrowprops={'fc':'black'})
        
figkw = {
    'title':'Cell Resource',
    'interval':50,
    'env_string':env,
    'event_string':events,
    'post_plot':[annot]
}
animkw = {'fig_conf':{'figsize':(8,6), 'dpi':100}}

html = expr.animate(figkw=figkw, animkw=animkw).to_html5_video()
plt.close()
HTML(html)